In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port','0'). \
config("spark.sql.warehouse.dir", f"/user/itv014453/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
!hadoop fs -head /public/trendytech/datasets/cust_transf.csv

1001,2023-05-15,1001,49.99
1002,2023-05-16,1002,29.99
1003,2023-05-17,1003,39.99
1004,2023-05-18,1004,19.99
1005,2023-05-19,1005,24.99
1001,2023-05-20,1002,29.99
1002,2023-05-21,1003,39.99
1003,2023-05-22,1004,19.99
1004,2023-05-23,1005,24.99
1005,2023-05-24,1001,49.99
1001,2023-05-25,1003,39.99
1002,2023-05-26,1004,19.99
1003,2023-05-27,1005,24.99
1004,2023-05-28,1001,49.99
1005,2023-05-29,1002,29.99
1001,2023-05-30,1003,39.99
1002,2023-05-31,1004,19.99
1003,2023-06-01,1005,24.99
1004,2023-06-02,1001,49.99
1005,2023-06-03,1002,29.99
1001,2023-06-04,1003,39.99
1002,2023-06-05,1004,19.99
1003,2023-06-06,1005,24.99
1004,2023-06-07,1001,49.99
1005,2023-06-08,1002,29.99
1006,2023-06-01,1001,49.99
1007,2023-06-02,1002,29.99
1008,2023-06-03,1003,39.99
1009,2023-06-04,1004,19.99
1010,2023-06-05,1005,24.99
1006,2023-06-06,1002,29.99
1007,2023-06-07,1003,39.99
1008,2023-06-08,1004,19.99
1009,2023-06-09,1005,24.99
1010,2023-06-10,1001,49.99
1006,2023-06-11,1003,39.99
1007,2023-06-12,1004,19.99
1

In [3]:
schema = 'customer_id integer,purchase_date date,product_id integer,amount float'

In [4]:
cust_trans_df = spark.read.schema(schema). \
option("dateFormat","yyyy-MM-dd"). \
csv("/public/trendytech/datasets/cust_transf.csv")

In [5]:
cust_trans_df.show()

+-----------+-------------+----------+------+
|customer_id|purchase_date|product_id|amount|
+-----------+-------------+----------+------+
|       1001|   2023-05-15|      1001| 49.99|
|       1002|   2023-05-16|      1002| 29.99|
|       1003|   2023-05-17|      1003| 39.99|
|       1004|   2023-05-18|      1004| 19.99|
|       1005|   2023-05-19|      1005| 24.99|
|       1001|   2023-05-20|      1002| 29.99|
|       1002|   2023-05-21|      1003| 39.99|
|       1003|   2023-05-22|      1004| 19.99|
|       1004|   2023-05-23|      1005| 24.99|
|       1005|   2023-05-24|      1001| 49.99|
|       1001|   2023-05-25|      1003| 39.99|
|       1002|   2023-05-26|      1004| 19.99|
|       1003|   2023-05-27|      1005| 24.99|
|       1004|   2023-05-28|      1001| 49.99|
|       1005|   2023-05-29|      1002| 29.99|
|       1001|   2023-05-30|      1003| 39.99|
|       1002|   2023-05-31|      1004| 19.99|
|       1003|   2023-06-01|      1005| 24.99|
|       1004|   2023-06-02|      1

In [6]:
cust_trans_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- purchase_date: date (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- amount: float (nullable = true)



In [7]:
cached_filtered_cust_trans_df = cust_trans_df.filter((cust_trans_df.purchase_date>="2023-05-01") & (cust_trans_df.purchase_date<"2023-06-08")).cache()

In [ ]:
aggf_cust_trans_df = cached_filtered_cust_trans_df.groupBy("product_id").sum("amount").withColumnRenamed("sum(amount)","revenue")

In [ ]:
top_selling_products = aggf_cust_trans_df.sort("revenue", ascending= False).limit(10).show()

In [ ]:
aggf_custc_trans_df = cached_filtered_cust_trans_df.groupBy("customer_id").sum("amount").withColumnRenamed("sum(amount)","revenue")

In [ ]:
top_customers = aggf_custc_trans_df.sort("revenue", ascending =False).limit(10).show()

In [ ]:
#doing all the above using spark external table


In [ ]:
spark.sql("use customer_transactions")

In [ ]:
spark.sql("show tables").show(truncate = False)

In [ ]:
spark.sql("create database customer_transactions_itv014453")

In [ ]:
spark.sql("create table customer_transactions_itv014453.transaction_ext(customer_id integer,purchase_date date,product_id integer,transaction_amount double) using csv location '/public/trendytech/datasets/cust_transf.csv'")

In [ ]:
spark.sql("use customer_transactions_itv014453")

In [ ]:
spark.sql("show tables").show(truncate = False)

In [9]:
spark.sql("""SELECT product_id,SUM(transaction_amount) AS revenue FROM customer_transactions_itv014453.transaction_ext
WHERE purchase_date >= DATE '2023-05-01' AND purchase_date <= DATE '2023-06-08'
GROUP BY product_id
ORDER BY revenue DESC LIMIT 10""").show()

+----------+--------------------+
|product_id|             revenue|
+----------+--------------------+
|      1003| 5.725592243903786E8|
|      1001| 5.566826411928239E8|
|      1002| 4.293836243948648E8|
|      1004|2.8620802440276194E8|
|      1005|2.7828564120213836E8|
|      1015|  12537.909999999963|
|      1014|  11492.909999999963|
|      1013|  10447.909999999963|
|      1012|   9402.909999999965|
|      1011|   8357.909999999967|
+----------+--------------------+



In [10]:
spark.sql("""SELECT customer_id,SUM(transaction_amount) AS revenue FROM customer_transactions_itv014453.transaction_ext
WHERE purchase_date >= DATE '2023-05-01' AND purchase_date <= DATE '2023-06-08'
GROUP BY customer_id
ORDER BY revenue DESC LIMIT 10""").show()

+-----------+--------------------+
|customer_id|             revenue|
+-----------+--------------------+
|       1001| 3.180884580005335E8|
|       1004| 3.101342580008687E8|
|       1005| 2.624090580015123E8|
|       1003|2.1468385800145328E8|
|       1002|2.0672965800144076E8|
|       1011|1.2724374111049211E8|
|       1006|1.2723851611049213E8|
|       1012| 1.113363861104608E8|
|       1007|1.1133116111046082E8|
|       1013| 9.542903111041903E7|
+-----------+--------------------+



In [11]:
spark.sql("cache table customer_transactions_itv014453.transaction_ext")

""


In [12]:
spark.sql("""SELECT product_id,SUM(transaction_amount) AS revenue FROM customer_transactions_itv014453.transaction_ext
WHERE purchase_date >= DATE '2023-05-01' AND purchase_date <= DATE '2023-06-08'
GROUP BY product_id
ORDER BY revenue DESC LIMIT 10""").show()

+----------+--------------------+
|product_id|             revenue|
+----------+--------------------+
|      1003| 5.725592243903785E8|
|      1001|  5.56682641192824E8|
|      1002| 4.293836243948648E8|
|      1004| 2.862080244027619E8|
|      1005|2.7828564120213836E8|
|      1015|  12537.909999999963|
|      1014|  11492.909999999963|
|      1013|  10447.909999999963|
|      1012|   9402.909999999965|
|      1011|   8357.909999999967|
+----------+--------------------+



In [13]:
spark.sql("""SELECT customer_id,SUM(transaction_amount) AS revenue FROM customer_transactions_itv014453.transaction_ext
WHERE purchase_date >= DATE '2023-05-01' AND purchase_date <= DATE '2023-06-08'
GROUP BY customer_id
ORDER BY revenue DESC LIMIT 10""").show()

+-----------+--------------------+
|customer_id|             revenue|
+-----------+--------------------+
|       1001| 3.180884580005336E8|
|       1004| 3.101342580008687E8|
|       1005|2.6240905800151232E8|
|       1003|2.1468385800145325E8|
|       1002|2.0672965800144082E8|
|       1011|1.2724374111049211E8|
|       1006|1.2723851611049214E8|
|       1012|1.1133638611046083E8|
|       1007|1.1133116111046082E8|
|       1013| 9.542903111041903E7|
+-----------+--------------------+

